In [1]:
!pip install --upgrade google-api-python-client

In [2]:
from googleapiclient.discovery import build
import pandas as pd
import numpy as np
from IPython.display import JSON
import json

In [3]:
api_key = "your api key"

In [4]:
channel_ids = ["UCOfzpYAxa1n6AJUbga2v07w",
               # yana kanallar bo'lishi mumkin
               ]

In [5]:
api_service_name = "youtube"
api_version = "v3"

youtube = build(
    api_service_name, api_version, developerKey=api_key)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=",".join(channel_ids)
)
response = request.execute()

print(json.dumps(response, indent=4))

{
    "kind": "youtube#channelListResponse",
    "etag": "9pWT0QUzHJkH653_APSW32RTr-Y",
    "pageInfo": {
        "totalResults": 1,
        "resultsPerPage": 5
    },
    "items": [
        {
            "kind": "youtube#channel",
            "etag": "dSl0rIqH80wsV4XG853vcF1anYY",
            "id": "UCOfzpYAxa1n6AJUbga2v07w",
            "snippet": {
                "title": "Mutolaa",
                "description": "Yoshlar ishlari agentligi va Xalq banki hamkorligidagi \u201cMutolaa\u201d loyihasining rasmiy YouTube sahifasiga xush kelibsiz!\n\u201cMutolaa\u201d \u2013 eng sara kitoblarning elektron, audio hamda video shaklidagi sifatli, tizimli platformasi! \n",
                "customUrl": "@mutolaaxona",
                "publishedAt": "2023-10-06T05:56:20.665622Z",
                "thumbnails": {
                    "default": {
                        "url": "https://yt3.ggpht.com/gizztbrfXlrAWpd1gZNp7WaWsFepx2sArXTuPgGDLSJ1phEQi5yIv-qqsWDiqLKfeRYrRyZY=s88-c-k-c0x00ffffff-no-rj"

In [6]:
def get_channel_stats(youtube, channel_ids):

  all_data = []

  request = youtube.channels().list(
      part="snippet, contentDetails, statistics",
      id=','.join(channel_ids)
  )
  response = request.execute()

  for item in response['items']:
    data = {
        'channelName': item['snippet']['title'],
        'subscribers': item['statistics']['subscriberCount'],
        'views': item['statistics']['viewCount'],
        'totalVideos': item['statistics']['videoCount'],
        'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
    }

    all_data.append(data)

  return (pd.DataFrame(all_data))

In [7]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [8]:
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Mutolaa,3430,74834,194,UUOfzpYAxa1n6AJUbga2v07w


In [9]:
request = youtube.playlistItems().list(
  part="snippet, contentDetails",
  playlistId="UUOfzpYAxa1n6AJUbga2v07w"
)
response = request.execute()

print(json.dumps(response, indent=4))

{
    "kind": "youtube#playlistItemListResponse",
    "etag": "5T2WvAVKmPyCGzIgBHLl9JE3Ptw",
    "nextPageToken": "EAAafVBUOkNBVWlFREV5UlVaQ00wSXhRelUzUkVVMFJURW9BVWpoLVkzVzR0YUZBMUFCV2pnaVEyaG9WbFpWT1cxbGJrSmFVVmhvYUUxWE5ESlJWWEJXV1cxa2FFMXVXWGRPTTJOVFJFRnFUbkp3ZFhoQ2FFUnZiV0Y1VkVGbkln",
    "items": [
        {
            "kind": "youtube#playlistItem",
            "etag": "1f-uVumkRZfgjHeODWZnaSqUOh0",
            "id": "VVVPZnpwWUF4YTFuNkFKVWJnYTJ2MDd3Lm53OXppZmhPTlVZ",
            "snippet": {
                "publishedAt": "2024-04-22T10:49:51Z",
                "channelId": "UCOfzpYAxa1n6AJUbga2v07w",
                "title": "Bir vafolik zaifaning xususida | Hikoya \u211673 | Mutolaa",
                "description": "Muallif: Mahmudxo\u2018ja Behbudiy\nO\u02bbqidi: Nargiza Mirzayeva\n\nMazkur ertak \u201cMutolaa\u201d loyihasi doirasida tayyorlandi.  Undan  tijoriy maqsadlarda foydalanish taqiqlanadi. Ko\u02bbchirilganda manba ko\u02bbrsatilishi shart.\n\n\ud83d\udcda O\u02bbz

In [10]:
playlist_id = "UUOfzpYAxa1n6AJUbga2v07w"

def get_video_ids(youtube, playlist_id):

  video_ids = []

  request = youtube.playlistItems().list(
    part="snippet, contentDetails",
    playlistId=playlist_id,
    maxResults = 50
  )
  response = request.execute()

  for item in response['items']:
    video_ids.append(item['contentDetails']['videoId'])

  next_page_token = response.get('nextPageToken')

  while next_page_token is not None:
    request = youtube.playlistItems().list(
      part="contentDetails",
      playlistId=playlist_id,
      maxResults = 50,
      pageToken = next_page_token
    )
    response = request.execute()
    for item in response['items']:
      video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')

  return video_ids

In [11]:
video_ids = get_video_ids(youtube, playlist_id)

In [12]:
len(video_ids)

192

In [13]:
def get_video_details(youtube, video_ids):

    all_video_info = []

    for i in range(0, len(video_ids), 50):

        request = youtube.videos().list(
          part="snippet, contentDetails, statistics",
          id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            stats_to_keep = {
                'snippet': ['title', 'description', 'publishedAt'],
                'statistics': ['viewCount', 'likeCount', 'commentCount'],
                'contentDetails': ['duration']
            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                      video_info[v] = video[k][v]
                    except:
                      video_info[v] = None

            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

In [14]:
video_df = get_video_details(youtube, video_ids)

In [15]:
video_df

,video_id,title,description,publishedAt,viewCount,likeCount,commentCount,duration
0,nw9zifhONUY,Bir vafolik zaifaning xususida | Hikoya №73 | ...,Muallif: Mahmudxo‘ja Behbudiy\nOʻqidi: Nargiza...,2024-04-22T10:49:51Z,180,8,0,PT1H11M2S
1,xYp2-iXNmVE,Javlon Jovliev - MUTOLAA VAQTI KELDI!,,2024-04-19T15:39:14Z,108,25,4,PT1M1S
2,SN6F8XRVlg8,Maktub | Hikoya №72 | Mutolaa,Muallif: Karimberdi To‘ramurod\nOʻqidi: Nargiz...,2024-04-19T12:00:26Z,216,10,1,PT18M2S
3,gAc5UL5MAqs,Quduq | Hikoya №71 | Mutolaa,Muallif: Abduqayom Yo‘ldosh\nOʻqidi: Nargiza M...,2024-04-18T13:30:23Z,284,7,1,PT16M58S
4,IL4N9kNzpow,Qotillar | Hikoya №69 | Mutolaa,Muallif: Ernest Heminguey\nOʻqidi: Boir Xolmir...,2024-04-18T12:30:09Z,271,17,1,PT23M58S
...,...,...,...,...,...,...,...,...
187,iyA1rjkmodw,Abdulla Avloniy | Mutolaa,Jadidlar: Abdulla Avloniy\nMuallif: Olim Oltin...,2023-11-05T15:51:39Z,206,11,0,PT2H51M58S
188,KGsMzM1gWXs,Gʻulom Zafariy | Mutolaa,Jadidlar: Gʻulom Zafariy\nMuallif: Olim Usmono...,2023-11-05T15:45:16Z,255,16,3,PT2H48M30S
189,dm_6yC6k2Iw,Munavvar qori Aburashidxonov | Mutolaa,Jadidlar: Munavvar qori Aburashidxonov\nMualli...,2023-10-31T13:53:18Z,1217,47,8,PT2H33M32S
190,FY5iEO0pUcU,Is’hoqxon to‘ra Ibrat | Mutolaa,Jadidlar: Isʼhoqxon toʻra Ibrat\nMuallif: Ulug...,2023-10-28T19:13:56Z,461,21,3,PT3H22M47S


In [16]:
video_df['duration']

0       PT1H11M2S
1          PT1M1S
2         PT18M2S
3        PT16M58S
4        PT23M58S
          ...    
187    PT2H51M58S
188    PT2H48M30S
189    PT2H33M32S
190    PT3H22M47S
191    PT2H48M38S
Name: duration, Length: 192, dtype: object

In [17]:
video_df['duration'] = video_df['duration'].apply(lambda x: x.replace('PT', ''))
video_df['duration']

0       1H11M2S
1          1M1S
2         18M2S
3        16M58S
4        23M58S
         ...   
187    2H51M58S
188    2H48M30S
189    2H33M32S
190    3H22M47S
191    2H48M38S
Name: duration, Length: 192, dtype: object

In [18]:
video_df['duration'] = video_df['duration'].apply(lambda x: x.replace('H', ':'))
video_df['duration']

0       1:11M2S
1          1M1S
2         18M2S
3        16M58S
4        23M58S
         ...   
187    2:51M58S
188    2:48M30S
189    2:33M32S
190    3:22M47S
191    2:48M38S
Name: duration, Length: 192, dtype: object

In [19]:
video_df['duration'] = video_df['duration'].apply(lambda x: x.replace('M', ':'))
video_df['duration']

0       1:11:2S
1          1:1S
2         18:2S
3        16:58S
4        23:58S
         ...   
187    2:51:58S
188    2:48:30S
189    2:33:32S
190    3:22:47S
191    2:48:38S
Name: duration, Length: 192, dtype: object

In [20]:
video_df['duration'] = video_df['duration'].apply(lambda x: x.replace('S', ''))
video_df['duration']

0       1:11:2
1          1:1
2         18:2
3        16:58
4        23:58
        ...   
187    2:51:58
188    2:48:30
189    2:33:32
190    3:22:47
191    2:48:38
Name: duration, Length: 192, dtype: object

In [21]:
import re
j = 0
oqidi = []
for i in video_df.description:

  match = re.search(r'ʻqidi:\s*(.*?)\n', i)

  if match:
      oqidi_text = match.group(1)
      print(f"{j}. Oʻqidi:", oqidi_text)
      oqidi.append("Oʻqidi:"+oqidi_text)

  else:
      print(f"{j}. Text not found")
      oqidi.append(None)
  j+=1

0. Oʻqidi: Nargiza Mirzayeva
1. Text not found
2. Oʻqidi: Nargiza Mirzayeva
3. Oʻqidi: Nargiza Mirzayeva
4. Oʻqidi: Boir Xolmirzayev
5. Oʻqidi: Iroda Axmedova
6. Text not found
7. Oʻqidi: Boir Xolmirzayev
8. Oʻqidi: Nargiza Mirzayeva
9. Text not found
10. Oʻqidi: Nargiza Mirzayeva
11. Oʻqidi: Iroda Axmedova
12. Oʻqidi: Nozimjon Nazarov
13. Oʻqidi: Nargiza Mirzayeva
14. Oʻqidi: Nozimjon Nazarov
15. Oʻqidi: Nargiza Ahmedova
16. Oʻqidi: Boir Xolmirzayev
17. Oʻqidi: Boir Xolmirzayev
18. Text not found
19. Oʻqidi: Abdulxay Otaxoʻjayev
20. Text not found
21. Oʻqidi: Abdulxay Otaxoʻjayev
22. Text not found
23. Text not found
24. Text not found
25. Text not found
26. Text not found
27. Oʻqidi: Boir Xolmirzayev
28. Oʻqidi: Ramziddin Sayfiyev
29. Oʻqidi: Boir Xolmirzayev
30. Oʻqidi: Iroda Ahmedova
31. Text not found
32. Oʻqidi: Iroda Ahmedova
33. Oʻqidi: Iroda Ahmedova
34. Text not found
35. Oʻqidi: Iroda Ahmedova
36. Oʻqidi: Iroda Ahmedova
37. Oʻqidi: Iroda Ahmedova
38. Oʻqidi: Nozimjon Nazarov

In [22]:
video_df['oqidi'] = pd.DataFrame(oqidi)
video_df

,video_id,title,description,publishedAt,viewCount,likeCount,commentCount,duration,oqidi
0,nw9zifhONUY,Bir vafolik zaifaning xususida | Hikoya №73 | ...,Muallif: Mahmudxo‘ja Behbudiy\nOʻqidi: Nargiza...,2024-04-22T10:49:51Z,180,8,0,1:11:2,Oʻqidi:Nargiza Mirzayeva
1,xYp2-iXNmVE,Javlon Jovliev - MUTOLAA VAQTI KELDI!,,2024-04-19T15:39:14Z,108,25,4,1:1,None
2,SN6F8XRVlg8,Maktub | Hikoya №72 | Mutolaa,Muallif: Karimberdi To‘ramurod\nOʻqidi: Nargiz...,2024-04-19T12:00:26Z,216,10,1,18:2,Oʻqidi:Nargiza Mirzayeva
3,gAc5UL5MAqs,Quduq | Hikoya №71 | Mutolaa,Muallif: Abduqayom Yo‘ldosh\nOʻqidi: Nargiza M...,2024-04-18T13:30:23Z,284,7,1,16:58,Oʻqidi:Nargiza Mirzayeva
4,IL4N9kNzpow,Qotillar | Hikoya №69 | Mutolaa,Muallif: Ernest Heminguey\nOʻqidi: Boir Xolmir...,2024-04-18T12:30:09Z,271,17,1,23:58,Oʻqidi:Boir Xolmirzayev
...,...,...,...,...,...,...,...,...,...
187,iyA1rjkmodw,Abdulla Avloniy | Mutolaa,Jadidlar: Abdulla Avloniy\nMuallif: Olim Oltin...,2023-11-05T15:51:39Z,206,11,0,2:51:58,Oʻqidi:Elbek Mirzohidov
188,KGsMzM1gWXs,Gʻulom Zafariy | Mutolaa,Jadidlar: Gʻulom Zafariy\nMuallif: Olim Usmono...,2023-11-05T15:45:16Z,255,16,3,2:48:30,Oʻqidi:Elbek Mirzohidov
189,dm_6yC6k2Iw,Munavvar qori Aburashidxonov | Mutolaa,Jadidlar: Munavvar qori Aburashidxonov\nMualli...,2023-10-31T13:53:18Z,1217,47,8,2:33:32,Oʻqidi:Abdulhay Otaxoʻjayev
190,FY5iEO0pUcU,Is’hoqxon to‘ra Ibrat | Mutolaa,Jadidlar: Isʼhoqxon toʻra Ibrat\nMuallif: Ulug...,2023-10-28T19:13:56Z,461,21,3,3:22:47,Oʻqidi:Muhammadjon Xoʻjayev


In [23]:
video_df['link'] = video_df['video_id'].apply(lambda x: 'https://www.youtube.com/watch?v='+str(x))

In [24]:
video_df = video_df.drop(['description'], axis=1)

In [25]:
video_df = video_df[['title', 'oqidi', 'duration','link', 'publishedAt', 'viewCount',]]
video_df

,title,oqidi,duration,link,publishedAt,viewCount
0,Bir vafolik zaifaning xususida | Hikoya №73 | ...,Oʻqidi:Nargiza Mirzayeva,1:11:2,https://www.youtube.com/watch?v=nw9zifhONUY,2024-04-22T10:49:51Z,180
1,Javlon Jovliev - MUTOLAA VAQTI KELDI!,None,1:1,https://www.youtube.com/watch?v=xYp2-iXNmVE,2024-04-19T15:39:14Z,108
2,Maktub | Hikoya №72 | Mutolaa,Oʻqidi:Nargiza Mirzayeva,18:2,https://www.youtube.com/watch?v=SN6F8XRVlg8,2024-04-19T12:00:26Z,216
3,Quduq | Hikoya №71 | Mutolaa,Oʻqidi:Nargiza Mirzayeva,16:58,https://www.youtube.com/watch?v=gAc5UL5MAqs,2024-04-18T13:30:23Z,284
4,Qotillar | Hikoya №69 | Mutolaa,Oʻqidi:Boir Xolmirzayev,23:58,https://www.youtube.com/watch?v=IL4N9kNzpow,2024-04-18T12:30:09Z,271
...,...,...,...,...,...,...
187,Abdulla Avloniy | Mutolaa,Oʻqidi:Elbek Mirzohidov,2:51:58,https://www.youtube.com/watch?v=iyA1rjkmodw,2023-11-05T15:51:39Z,206
188,Gʻulom Zafariy | Mutolaa,Oʻqidi:Elbek Mirzohidov,2:48:30,https://www.youtube.com/watch?v=KGsMzM1gWXs,2023-11-05T15:45:16Z,255
189,Munavvar qori Aburashidxonov | Mutolaa,Oʻqidi:Abdulhay Otaxoʻjayev,2:33:32,https://www.youtube.com/watch?v=dm_6yC6k2Iw,2023-10-31T13:53:18Z,1217
190,Is’hoqxon to‘ra Ibrat | Mutolaa,Oʻqidi:Muhammadjon Xoʻjayev,3:22:47,https://www.youtube.com/watch?v=FY5iEO0pUcU,2023-10-28T19:13:56Z,461


In [26]:
video_df.to_csv('Mutolaa+link.csv')